In [30]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier

In [5]:
df = pd.read_csv("Default.csv")
df["default"] = df["default"].astype("category")
df["student"] = df["student"].astype("category")

In [38]:
y = (df["default"] == "Yes").values
df["student_dummy"] = (df["student"] == "Yes").astype(int)
columnas_posibles = ["balance", "income", "student_dummy"]
x = df[columnas_posibles]
columnas_posibles = x.columns

## Accuracy LogisticRegresion

In [17]:
X = df[columnas_posibles]

In [18]:
log_clf = make_pipeline(StandardScaler(),
                        LogisticRegression(penalty='l2', C=1, solver='lbfgs',
                                           max_iter=1000, random_state=42))

In [20]:
accuracy_cv = cross_val_score(log_clf, X, y, scoring="accuracy", cv=5)

print("Accuracy media:", accuracy_cv.mean())
print("Desviación estándar:", accuracy_cv.std())

Accuracy media: 0.9732
Desviación estándar: 0.001568438714135832


#Punto extra

In [21]:
auc_cv = cross_val_score(log_clf, X, y, scoring="roc_auc_ovr", cv=5)

print("AUC media:", auc_cv.mean())
print("Desviación estándar:", auc_cv.std())

AUC media: 0.9495737380053795
Desviación estándar: 0.009607585904774351


#Accuracy de Bagging

In [40]:
B = 500
n = len(df)

all_preds = []

for b in range(B):

    cols_elegidas = np.random.choice(columnas_posibles, size=2, replace=True)

    X_full = df[cols_elegidas].values

    idx = np.random.choice(n, n, replace=True)
    X_b = X_full[idx]
    y_b = y[idx]

    tree = DecisionTreeClassifier()
    tree.fit(X_b, y_b)

    y_pred_full = tree.predict(X_full)

    all_preds.append(y_pred_full)


preds_matrix = np.vstack(all_preds)

y_pred_bagging = (preds_matrix.mean(axis=0) >= 0.5).astype(int)

accuracy_bagging = np.mean(y_pred_bagging == y)

print("Accuracy del modelo bagging :", accuracy_bagging)

Accuracy del modelo bagging : 1.0
